In [ ]:
%matplotlib inline

import sys
import rospy
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from geometry_msgs.msg import WrenchStamped
from pathlib import Path
from pprint import pprint
from scikitMedicalRobot.ros import medical_robot_helper, tf2_helper
from scikitMedicalRobot.ros.moveit_helper import robot_control
from scikitMedicalRobot.calibration.tool import trajectory_recorder, ftsensor_calibration_recorder
from scikitMedicalRobot.calibration.tool import ftsensor_calibration
from scikitMedicalRobot.utilities import generate_timedate_cache_file

Definition: environment

In [ ]:
sim_experiment = True
jump_threshold = 4            # cartesian plan jump threshold

Definition: link name

In [ ]:
base_link_name = 'base_link'
ee_link_name = 'tool0'
tool_link_name = 'tool_link'
medical_image_link = 'medical_link'
tool_rotation_axis = '-y'

`scikitMedicalRobot` Toolkits

In [ ]:
rospy.init_node('ftsensor_calibration')

th = tf2_helper()
mrh = medical_robot_helper(tool_link_name, tool_rotation_axis, ee_link_name, base_link_name, medical_image_link)

Robot Control

In [ ]:
rc = robot_control(argv=sys.argv, group_name='manipulator', ee_link_name='tool0')

if not sim_experiment:
    rc.plan_conf_max_a = 0.04
    rc.plan_conf_max_v = 0.04
    rc.move_group.limit_max_cartesian_link_speed(0.005)
else:
    rc.plan_conf_max_a = 0.6
    rc.plan_conf_max_v = 0.6
    rc.move_group.clear_max_cartesian_link_speed()

Functions for sample ft sensor and trans

In [ ]:
N = 100
def sample_ftsensor():
    datas = [[0.0] * 6] * N
    for i in range(N):
        wrench_data = rospy.wait_for_message('/ft_sensor_filtered', WrenchStamped, timeout=10)   
        datas[i][0] = wrench_data.wrench.force.x
        datas[i][1] = wrench_data.wrench.force.y
        datas[i][2] = wrench_data.wrench.force.z
        datas[i][3] = wrench_data.wrench.torque.x
        datas[i][4] = wrench_data.wrench.torque.y
        datas[i][5] = wrench_data.wrench.torque.z
    return np.mean(datas, axis=0)

def sample_trans():
    t = th.sample(ref_link='base_link', target_link='tool0')
    return th.matrix_from_transform(t.transform)

pprint(f'ft sensor: {sample_ftsensor()}')
pprint(f'trans: {sample_trans()}')

Manual Calibration

In [ ]:
recorder = ftsensor_calibration_recorder(sample_ftsensor, sample_trans)
recorder.run()

Auto Calibration

In [ ]:
start_joints_position = rc.move_group.get_current_joint_values()

In [ ]:
# load trajectory_recorder data
with open('exp/trajectory_recorder/trajectory_recorder_20230103_134417_318093.pickle', 'rb') as f:
    joint_states = pickle.load(f)

trans_list = []
ft_list = []

for idx in tqdm(range(len(joint_states))):
    rc.move_group.go(joint_states[idx], wait=True)
    rc.move_group.stop()
    rospy.sleep(1.0)
    
    trans = sample_trans()
    ft = sample_ftsensor()
    if trans is not None and ft is not None:
        trans_list.append(trans)
        ft_list.append(ft)
        
rc.move_group.go(start_joints_position)
rc.move_group.stop()

f_name = generate_timedate_cache_file(caches_dir='exp', child_dir='ftsensor_recorder', filetype='pickle', prefix='ftsensor_recorder')
with open(f_name, "wb") as f:
    pickle.dump([np.array(trans_list), np.array(ft_list)], f)
print(f"saved to {f_name}")

Evaluation

In [ ]:
file_name = Path('exp/trajectory_recorder/trajectory_recorder_20230103_134417_318093.pickle')

In [ ]:
# calibration from dataset
with open(file_name, 'rb') as f:
    data = pickle.load(f)
    
ft_list = data[1]
trans_list = data[0]

result = ftsensor_calibration.calibration(ft_list, trans_list)
text = ftsensor_calibration.evaluate(np.array(ft_list), np.array(trans_list), result, no_plot=False)
print(text)

Try

In [ ]:
ftsensor_calibration.apply_compensation(sample_ftsensor(), sample_trans(), result)